In [1]:
import pandas as pd
data = pd.read_csv("../data/AAPL_enriched.csv", index_col=0, parse_dates=True)


In [2]:
import numpy as np

def backtest_sma(df, fast=10, slow=50):
    temp = df.copy()
    temp[f"SMA_{fast}"] = temp["Close"].rolling(fast).mean()
    temp[f"SMA_{slow}"] = temp["Close"].rolling(slow).mean()
    
    temp["Signal"] = 0
    temp.loc[temp[f"SMA_{fast}"] > temp[f"SMA_{slow}"], "Signal"] = 1
    temp.loc[temp[f"SMA_{fast}"] < temp[f"SMA_{slow}"], "Signal"] = -1
    temp["Position"] = temp["Signal"].shift(1).fillna(0)
    
    temp["Return"] = temp["Close"].pct_change()
    temp["Strategy_Return"] = temp["Position"] * temp["Return"]
    
    strat_curve = (1 + temp["Strategy_Return"]).cumprod()
    years = (temp.index[-1] - temp.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = temp["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (temp["Strategy_Return"].mean() * 252) / (temp["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    
    return {"Fast": fast, "Slow": slow, "CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}


In [3]:
results = []
for fast in [5, 10, 20]:
    for slow in [50, 100, 200]:
        if fast < slow:  # valid only when fast < slow
            res = backtest_sma(data, fast, slow)
            results.append(res)

results_df = pd.DataFrame(results)
results_df


C:\Users\adity\AppData\Local\Temp\ipykernel_3728\2862344019.py:20: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = (temp["Strategy_Return"].mean() * 252) / (temp["Strategy_Return"].std() * np.sqrt(252))
C:\Users\adity\AppData\Local\Temp\ipykernel_3728\2862344019.py:20: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = (temp["Strategy_Return"].mean() * 252) / (temp["Strategy_Return"].std() * np.sqrt(252))
C:\Users\adity\AppData\Local\Temp\ipykernel_3728\2862344019.py:20: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = (temp["Strategy_Return"].mean() * 252) / (temp["Strategy_Return"].std() * np.sqrt(252))
C:\Users\adity\AppData\Local\Temp\ipykernel_3728\2862344019.py:20: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = (temp["Strategy_Return"].mean() * 252) / (temp["Strategy_Return"].std() * np.sqrt(252))
C:\Users\adity\AppData\Local\Temp\ipykernel_3728\2862344019.py:20: RuntimeWarning: invalid value enc

,Fast,Slow,CAGR,Volatility,Sharpe,MaxDD
0,5,50,-0.121856,0.041243,-3.175536,-0.02633
1,5,100,0.000000,0.000000,NaN,0.00000
2,5,200,0.000000,0.000000,NaN,0.00000
3,10,50,-0.121856,0.041243,-3.175536,-0.02633
4,10,100,0.000000,0.000000,NaN,0.00000
5,10,200,0.000000,0.000000,NaN,0.00000
6,20,50,-0.121856,0.041243,-3.175536,-0.02633
7,20,100,0.000000,0.000000,NaN,0.00000
8,20,200,0.000000,0.000000,NaN,0.00000


In [4]:
best = results_df.sort_values("Sharpe", ascending=False).iloc[0]
print("Best Strategy by Sharpe:")
print(best)


Best Strategy by Sharpe:
Fast           5.000000
Slow          50.000000
CAGR          -0.121856
Volatility     0.041243
Sharpe        -3.175536
MaxDD         -0.026330
Name: 0, dtype: float64


In [5]:
import os
from datetime import datetime

os.makedirs("../logs", exist_ok=True)

results_df.to_csv("../logs/Day6_multi_strategy_results.csv", index=False)

with open("../logs/Day6_best_strategy.txt", "w") as f:
    f.write(f"Run date: {datetime.today().date()}\n")
    f.write(str(best))
    
print("Results saved to /logs/")


Results saved to /logs/
